In [1]:
from __future__ import annotations

import pathlib
from typing import Optional, Any

import transformers
import wandb
import torch 
import typer
import yaml
import torchdata.datapipes as dp

import audiocap.metrics
import audiocap.data
import audiocap.callbacks
import audiocap.models

from train_whisper_supervised import get_whisper_model


In [9]:
!CUDA_VISIBLE_DEVICES=3
!echo $CUDA_VISIBLE_DEVICES

In [76]:
load_checkpoint = pathlib.Path("../checkpoints/stoic-totem-29/checkpoint-18900")
dataset_dir = pathlib.Path("../../maratmp/audio-captioning/data/clotho_v2.1/audiofolder/test/")
output_dir = pathlib.Path("../inference_outputs")
generate_config = pathlib.Path("../configs/generate_clotho_test.yaml")


In [77]:
with open(generate_config, "r") as f:
    inference_config_dict: dict = yaml.safe_load(f)

In [78]:
generate_args_dict = inference_config_dict["generate_args"]
architecture_config = inference_config_dict["architecture"]
architecture_name = architecture_config["name"]
use_pretrained_encoder = architecture_config["use_pretrained_whisper_encoder"]
use_pretrained_decoder = architecture_config["use_pretrained_whisper_decoder"]

data_config = inference_config_dict["data"]
dataset_name = data_config["dataset_name"]
task = data_config["task"]
data_limit = data_config["data_limit"]


In [34]:
config = transformers.WhisperConfig.from_pretrained(architecture_name)
tokenizer = transformers.WhisperTokenizer.from_pretrained(architecture_name, language="en", task="transcribe")
feature_extractor = transformers.WhisperFeatureExtractor.from_pretrained(architecture_name)
assert isinstance(config, transformers.WhisperConfig)
model = get_whisper_model(architecture_name, config, load_checkpoint, use_pretrained_encoder, use_pretrained_decoder)

tuned_params = sum(p.shape.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.shape.numel() for p in model.parameters())
print(f"Number of trained parameters: {tuned_params}/{total_params} = {tuned_params/total_params*100:.2f}%")

Number of trained parameters: 1543304960/1543304960 = 100.00%


In [70]:
!ls ../../maratmp/audio-captioning/data/clotho_v2.1/audiofolder/development/

'00264 hill creek 1.wav'
 002_78_rpm_vinyl_noise_44_16_lossless.wav
'00294 harvest festival rumour 1.wav'
'00332 lake beach 1.wav'
'00523 trolley bus and city bus 1.wav'
'00708 christmas background 1.wav'
'008 War 2.wav'
 00_BRUSH.wav
'01052010 Tornado siren test.wav'
'0111 Ambulance.wav'
'0112_032412 bubbling brook 1min 96k.wav'
'01-21 Footsteps, sneakers on gravel and leaves.wav'
'01-31 Footsteps, Wood, Barefoot, Jumps &amp; Scuffs.wav'
'0168 Convenience_store.wav'
'01760 industrial noise.wav'
'01 A pug struggles to breathe 1_14_2008.wav'
'01 barreau bunker original.wav'
 01-BikeDemo_Speaker.wav
 01-bottle-water-sparkle.wav
'01 car sequence.wav'
'01 catsnoring.wav'
'01 hospital elevator with computer voice.wav'
 01_Lanes_Island_Ambient_48_24.wav
 01-Osterport_WaitingHall.wav
'01 residential elevator with door open and close.wav'
'_01storm - orage.wav'
 020220_00.wav
'0208 Fountain_Parque_del_Peru.wav'
'0211_165814 tea prep.wav'
'0211_170236 walk downstairs.wav'
'0221 Bar_terrace.wav'

In [87]:
# set eval dataset
audioset_dir = None
audiocaps_dir = None
clotho_dir = None

if dataset_name == "clotho":
    clotho_dir = dataset_dir
elif dataset_name == "audioset":
    audioset_dir = dataset_dir
elif dataset_name == "audiocaps":
    audiocaps_dir = dataset_dir
else:
    raise ValueError(f"Unknown dataset name: {dataset_name}")

dataset_weights = {dataset_name: 1}    
datasets_val_limits = {dataset_name: None}

dataset, audiofolders, ds_val_alternatives = audiocap.data.load_dataset_mixture(
    clotho_dir,
    audioset_dir,
    audiocaps_dir,
    dataset_weights,
    datasets_val_limits,
    0,
    0,
    tokenizer,
    feature_extractor,
)

ValueError: Expected object or value

In [81]:
model.generate(torch.rand((1280,80,3)), **generate_args_dict)

RuntimeError: The size of tensor a (2) must match the size of tensor b (1500) at non-singleton dimension 1